In [1]:
import numpy as np
from scipy.special import comb
from scipy.optimize import linear_sum_assignment
%matplotlib notebook
%matplotlib inline
import matplotlib.pyplot as plt

# Bethe Permanent

In [2]:
def permanent(A):
    assert(len(A.shape)==2),"Input should be a 2D matrix"
    assert(A.shape[0]==A.shape[1]),"Input should be a square matrix"
    
    perm=np.float128(0)
    n=A.shape[1]
    C=2**n
    
    for k in range(1,C):
        rowsumprod=np.float128(1)
        chi=np.array(list(np.binary_repr(k))).astype(np.float128)
        for m in range(0,n):
            rowsum=np.float128(0)
            
            for p in range(0,len(chi)):
                rowsum += chi[p]*A[m,len(chi)-p-1]
            
            rowsumprod *= rowsum
            
        perm += ((-1)**(n-chi.sum()))*rowsumprod
    
    return perm

In [3]:
def belief_prop(P,max_iter=50, eps=0.1):
    n=P.shape[0]
    phi=np.sqrt(P)
    m_x2y=np.ones((n,n,n),dtype=np.float128) 
    m_y2x=np.ones((n,n,n),dtype=np.float128) 
    m_x2y_new=np.ones((n,n,n),dtype=np.float128) 
    m_y2x_new=np.ones((n,n,n),dtype=np.float128) 
    belief_xy=np.zeros((n,n,n,n),dtype=np.float128)#belief_xy(i,j,xi,yj) is the value of b(x_i,y_j) for a given value of x_i and y_j
    belief_x=np.zeros((n,n),dtype=np.float128)
    belief_y=np.zeros((n,n),dtype=np.float128)
    
    
    for t in range(max_iter):
        
        #m_x2y_new
        for i in range(n):
            for j in range(n):
                for yj in range(n):
                    sum_xi=0             #sum over all x_i
                    for xi in range(n):
#                         print(i,j,xi,yj,not((xi==j)!=(yj==i)))
                        if not((xi==j)!=(yj==i)): # if psi(x_i,y_j) is non zero
                            prod_m_yk2xi=1     # prod_{k!=j}m_{y_k}(x_i)
                            for k in range(n):
                                if k!=j:
                                    prod_m_yk2xi*=m_y2x[k,i,xi]
                            sum_xi+=phi[i,xi]*prod_m_yk2xi  # the whole summation (over x_i)
                    m_x2y_new[i,j,yj]=sum_xi # message from x_i to y_j for a particular value of y_j
#                     print(i,j,yj,sum_xi)
        m_x2y_new=m_x2y_new/np.sum(m_x2y_new) #normalization
        
        #m_y2x_new
        for j in range(n):
            for i in range(n):
                for xi in range(n):
                    sum_yj=0            
                    for yj in range(n):        
                        if not((xi==j)!=(yj==i)): 
                            prod_m_xk2yj=1     
                            for k in range(n):
                                if k!=i:
                                    prod_m_xk2yj*=m_x2y[k,j,yj]
                            sum_yj+=phi[yj,j]*prod_m_xk2yj  
                    m_y2x_new[j,i,xi]=sum_yj         
        m_y2x_new=m_y2x_new/np.sum(m_y2x_new)#normalization
        
        
        #update using log rule
        for i in range(n):
            for j in range(n):
                for yj in range(n):
                    if m_x2y_new[i,j,yj] >0:
                        m_x2y[i,j,yj]=np.exp(np.log(m_x2y[i,j,yj]) + eps*(np.log(m_x2y_new[i,j,yj]) - np.log(m_x2y[i,j,yj])))
    #                   print("#",i,j,yj,(m_x2y[i,j,yj]),(m_x2y_new[i,j,yj]))
                    
        for j in range(n):
            for i in range(n):
                for xi in range(n):
                    if m_y2x_new[j,i,xi]>0:
                        m_y2x[j,i,xi]=np.exp(np.log(m_y2x[j,i,xi]) + eps*(np.log(m_y2x_new[j,i,xi]) - np.log(m_y2x[j,i,xi])))                   
                    
        m_x2y=m_x2y/np.sum(m_x2y) #normalization
        m_y2x=m_y2x/np.sum(m_y2x)#normalization

        
#         print(t, np.sum(np.abs(m_y2x-m_y2x_new)))

    #compute joint beliefs for each i,j and for each value of x_i and y_j
#     for i in range(n):
#         for j in range(n):
#             for xi in range(n):
#                 for yj in range(n):
#                     if not((xi==j)!=(yj==i)):
#                         prod_m_yk2xi=1     # prod_{k!=j}m_{y_k}(x_i)
#                         prod_m_xl2yj=1
#                         for k in range(n):
#                             if k!=j:
#                                 prod_m_yk2xi*=m_y2x[k,i,xi]
#                         for l in range(n):
#                             if l!=i:
#                                 prod_m_xl2yj*=m_x2y[l,j,yj]
#                         belief_xy[i,j,xi,yj]=phi[i,xi]*phi[yj,j]*prod_m_yk2xi*prod_m_xl2yj

                    
    
#     # normalize the joint beliefs                    
#     for i in range(n):
#         for j in range(n):
#             belief_xy[i,j,:,:]=belief_xy[i,j,:,:]/np.sum(belief_xy[i,j,:,:])
    
    #compute single node beliefs
    for i in range(n):
        for xi in range(n):
            prod_m_yk2xi=1
            for k in range(n):
                prod_m_yk2xi*=m_y2x[k,i,xi]
            belief_x[i,xi]=phi[i,xi]*prod_m_yk2xi
    
    for j in range(n):
        for yj in range(n):
            prod_m_xl2yj=1
            for l in range(n):
                prod_m_xl2yj*=m_x2y[l,j,yj]
            belief_y[j,yj]=phi[yj,j]*prod_m_xl2yj
            
    belief_xy=np.zeros((n,n,n,n),dtype=np.float128)
    for i in range(n):
        for j in range(n):
            for xi in range(n):
                for yj in range(n):
                    if not((xi==j)!=(yj==i)):
                        belief_xy[i,j,xi,yj]=belief_x[i,xi]*belief_y[j,yj]/(m_x2y[i,j,yj]*m_y2x[j,i,xi])
                    else:
                        belief_xy[i,j,xi,yj]=0
    
    for i in range(n):
        for j in range(n):
            belief_xy[i,j,:,:]=belief_xy[i,j,:,:]/np.sum(belief_xy[i,j,:,:])
    
    #normalize single node beliefs
    for i in range(n):
        belief_x[i,:]=belief_x[i,:]/np.sum(belief_x[i,:])
            
    for j in range(n):
        belief_y[j,:]=belief_y[j,:]/np.sum(belief_y[j,:])
        
#     belief_xy=belief_xy/n
#     belief_x=belief_x/(n)
#     belief_y=belief_y/(n)
        
    return m_x2y,m_y2x,belief_xy,belief_x,belief_y,(np.sum(np.abs(m_y2x-m_y2x_new))<1e-10),np.sum(np.abs(m_y2x-m_y2x_new))

In [4]:
def bfe(P,belief_xy,belief_x,belief_y,m_x2y,m_y2x):
        
    t1=t2=t3=t4=t5=t6=0
    n=P.shape[0]
#     belief_x=np.zeros((n,n),dtype=float)
#     belief_y=np.zeros((n,n),dtype=float)
    phi=np.sqrt(P)
    
#     belief_xy=np.zeros((n,n,n,n),dtype=np.float128)
#     for i in range(n):
#         for j in range(n):
#             belief_xy[i,j,:,:]=np.outer(belief_x[i,:],belief_y[j,:])/(m_x2y[i,j]*m_y2x[j,i])
#     for i in range(n):
#         for j in range(n):
#             belief_xy[i,j,:,:]=belief_xy[i,j,:,:]/np.sum(belief_xy[i,j,:,:])

    
#     for i in range(n):
#         belief_x[i,:]=np.sum(belief_xy[i,0],axis=1)
#         belief_y[i,:]=np.sum(belief_xy[0,i],axis=0)
        
    for i in range(n):
        for j in range(n):
            for xi in range(n):
                for yj in range(n):
                    if belief_xy[i,j,xi,yj]!=0 and not((xi==j)!=(yj==i)):
                        t1+=belief_xy[i,j,xi,yj]*np.log(phi[i,xi]*phi[yj,j])
                        t2+=belief_xy[i,j,xi,yj]*np.log(belief_xy[i,j,xi,yj])
    
    for i in range(n):
        for xi in range(n):
            if belief_x[i,xi]!=0:
                t3+=belief_x[i,xi]*np.log(belief_x[i,xi])
                t5+=belief_x[i,xi]*np.log(phi[i,xi])
                                          
    for j in range(n):
        for yj in range(n):
            if belief_y[j,yj]!=0:
                t4+=belief_y[j,yj]*np.log(belief_y[j,yj])
                t6+=belief_y[j,yj]*np.log(phi[yj,j])
    
#     print('t1', -t1)    
#     print('t2', t2)
#     print('t3', -(n-1)*t3)
#     print('t4', -(n-1)*t4)
#     print('t5', (n-1)*t5)
#     print('t6', (n-1)*t6)
    energy=-t1+t2-(n-1)*t3-(n-1)*t4+(n-1)*t5+(n-1)*t6
#     print(energy)
    return np.exp(-energy)

In [5]:
def f_bp(P):
    
    def f_bp_p(beta):
        n=P.shape[1]
        z=0
#         print(beta.sum())
        beta=beta.reshape(n,n)
#         print(beta)
        for i in range(n):
            for j in range(n):
                
#                 if beta[i,j]<0:
#                     print(beta[i,j])
                if beta[i,j]!=0 and beta[i,j]!=1 and beta[i,j]>0 and P[i,j]>0: 
                    z+=beta[i,j]*np.log(beta[i,j]/P[i,j])-(1-beta[i,j])*np.log(1-beta[i,j])
#                 print((beta[i,j]/P[i,j]))
        return z
    return f_bp_p

# Experiments

#### Generate Random Matrices

In [45]:
def random_sparse_matrix(rng,n,kappa):
    mask=rng.binomial(1,kappa,(n,n))
    # mat=rng.exponential(size=(n,n))
    # mat=rng.uniform(size=(n,n))
    # mat=rng.gamma(2,size=(n,n))
    return np.multiply(mask,mat)


#### Compute permanent for each n with sparsity c/n

In [ ]:
rng=np.random.default_rng(5)

for c in np.linspace(1,3,num=9):
    eta_avg_arr=np.zeros(10)
    samples=10
    print('c: ',c)
    for n in range(3,10):
        eta_sum=0
        k=c/n
        print('n: ',n)
        for t in range(0,samples):
            P=random_sparse_matrix(rng,n,k)
            perm=permanent(P)
            
            while (np.isclose(perm,0)):
                P=random_sparse_matrix(rng,n,k)
                perm=permanent(P)

            print(P)
            m_x2y,m_y2x,belief_xy,belief_x,belief_y,conv,conv_value=belief_prop(P,300)
            perm_bethe=bfe(P,belief_xy,belief_x,belief_y,m_x2y,m_y2x)
            ub=perm_bethe*np.sqrt(2)**P.shape[0]
            eta_sum+=np.log(perm/perm_bethe)/P.shape[0]

            print("Perm: \t",perm)
            print("Bethe: \t" ,perm_bethe)
            print("Bounds: \t",perm_bethe<perm<ub)
            print("Convergence: \t",conv)
            if not conv:
                print("Conv Value: ", conv_value)
            print("Eta: \t",np.log(perm/perm_bethe)/P.shape[0])
            print('\n\n\n')
        eta_avg_arr[n]=eta_sum/samples
    np.save('eta_uniform_linear_c_'+str(c),eta_avg_arr)

#### Compute permanent for each $\kappa$ and n


In [ ]:
rng=np.random.default_rng(2)
for n in range(3,10):
    samples=10
    eta_avg_arr=np.zeros(10)
    idx=0
    for k in np.linspace(0,1,num=11)[1:10]:
        eta_sum=0
        print('k: ',k)
        for t in range(0,samples):
            P=random_sparse_matrix(rng,n,k)
            perm=permanent(P)
            
            while (np.isclose(perm,0)):
                P=random_sparse_matrix(rng,n,k)
                perm=permanent(P)

            # print(P)
            m_x2y,m_y2x,belief_xy,belief_x,belief_y,conv,conv_value=belief_prop(P,300)
            perm_bethe=bfe(P,belief_xy,belief_x,belief_y,m_x2y,m_y2x)
            ub=perm_bethe*np.sqrt(2)**P.shape[0]
            eta_sum+=np.log(perm/perm_bethe)/P.shape[0]

            print("Perm: \t",perm)
            print("Bethe: \t" ,perm_bethe)
            print("Bounds: \t",perm_bethe<perm<ub)
            print("Convergence: \t",conv)
            if not conv:
                print("Conv Value: ", conv_value)
            print("Eta: \t",np.log(perm/perm_bethe)/P.shape[0])
            print('\n\n\n')
        eta_avg_arr[idx]=eta_sum/samples
        idx+=1
    np.save('eta_avg_arr_gamma_n_'+str(n),eta_avg_arr)

## Results

* $\eta = (1/n)*log(perm/perm_B)$
* Each element of the matrix is either 0 (with probability 1-$\kappa$) or belongs to a distribution with pdf $g(x)$ (with probability $\kappa$)

* Experiment is conducted for nxn matrices where and $\kappa$ varies from 0.1 to 0.9.
* Experiment is conducted for cases when $g(x)$ is uniform, exponential(scale=1), gamma(shape=2,scale=1), standard normal
* For each $\kappa$ and n, eta is averaged over 10 

* values of $\eta_B$ when $g(x)$ is Uniform distribution

| $\kappa$ | 0.1    | 0.2    | 0.3    | 0.4    | 0.5    | 0.6    | 0.7    | 0.8    | 0.9    |
|----------|--------|--------|--------|--------|--------|--------|--------|--------|--------|
| n=3      | 0.0023 | 0.0048 | 0.0328 | 0.0484 | 0.0434 | 0.0843 | 0.0894 | 0.1468 | 0.2105 |
| n=4      | 0.0128 | 0.0290 | 0.0469 | 0.0627 | 0.0992 | 0.0927 | 0.1437 | 0.2146 | 0.2421 |
| n=5      | 0.0198 | 0.0235 | 0.0154 | 0.0640 | 0.0958 | 0.1130 | 0.1876 | 0.2094 | 0.2226 |
| n=6      | 0.0184 | 0.0353 | 0.0534 | 0.1075 | 0.1242 | 0.1299 | 0.1865 | 0.1960 | 0.2021 |
| n=7      | 0.0206 | 0.0271 | 0.0559 | 0.1104 | 0.1294 | 0.1588 | 0.1821 | 0.1820 | 0.1882 |
| n=8      | 0.0094 | 0.0172 | 0.0981 | 0.1228 | 0.1406 | 0.1585 | 0.1696 | 0.1716 | 0.1751 |
| n=9      | 0.0200 | 0.0628 | 0.0801 | 0.1262 | 0.1345 | 0.1520 | 0.1564 | 0.1611 | 0.1629 |

* values of $\eta_B$ when $g(x)$ is Exponential distribution (scale=1)

| $\kappa$ | 0.1    | 0.2    | 0.3    | 0.4    | 0.5    | 0.6    | 0.7    | 0.8    | 0.9    |
|----------|--------|--------|--------|--------|--------|--------|--------|--------|--------|
| n=3      | 0.0036 | 0.0378 | 0.0143 | 0.0400 | 0.0312 | 0.1051 | 0.0670 | 0.1053 | 0.1108 |
| n=4      | 0.0080 | 0.0039 | 0.0083 | 0.0522 | 0.0474 | 0.1003 | 0.1701 | 0.1615 | 0.1659 |
| n=5      | 0.0037 | 0.0244 | 0.0171 | 0.0605 | 0.0742 | 0.1346 | 0.1292 | 0.1627 | 0.2075 |
| n=6      | 0.0056 | 0.0180 | 0.0120 | 0.0626 | 0.1063 | 0.1508 | 0.1631 | 0.1812 | 0.1957 |
| n=7      | 0.0037 | 0.0184 | 0.0370 | 0.0704 | 0.1250 | 0.1573 | 0.1644 | 0.1736 | 0.1785 |
| n=8      | 0.0102 | 0.0438 | 0.0646 | 0.0990 | 0.1165 | 0.1521 | 0.1557 | 0.1623 | 0.1692 |
| n=9      | 0.0172 | 0.0422 | 0.0782 | 0.1152 | 0.1203 | 0.1447 | 0.1542 | 0.1566 | 0.1590 |

* values of $\eta_B$ when $g(x)$ is Gamma distribution (shape=2,scale=1)

| $\kappa$ | 0.1    | 0.2    | 0.3    | 0.4    | 0.5    | 0.6    | 0.7    | 0.8    | 0.9    |
|----------|--------|--------|--------|--------|--------|--------|--------|--------|--------|
| n=3      | 0.0027 | 0.0229 | 0.0074 | 0.0690 | 0.0590 | 0.0800 | 0.1096 | 0.1448 | 0.2173 |
| n=4      | 0.0181 | 0.0089 | 0.0314 | 0.0421 | 0.1075 | 0.1410 | 0.1681 | 0.1475 | 0.2227 |
| n=5      | 0.0194 | 0.0162 | 0.0456 | 0.0551 | 0.0986 | 0.1625 | 0.1742 | 0.1947 | 0.2236 |
| n=6      | 0.0025 | 0.0605 | 0.0685 | 0.0768 | 0.1133 | 0.1650 | 0.1741 | 0.1947 | 0.2003 |
| n=7      | 0.0089 | 0.0521 | 0.1134 | 0.1064 | 0.1382 | 0.1767 | 0.1760 | 0.1847 | 0.1881 |
| n=8      | 0.0042 | 0.0416 | 0.0773 | 0.1183 | 0.1463 | 0.1581 | 0.1655 | 0.1725 | 0.1749 |


---


$\kappa$ = c/n : for a matrix of size n$\times$n sparsity factor is linear in n

* values of $\eta_B$ when $g(x)$ is Uniform distribution

| n      | 3      | 4      | 5      | 6      | 7      | 8      | 9      |
|--------|--------|--------|--------|--------|--------|--------|--------|
| c=1.00 | 0.0219 | 0.0280 | 0.0134 | 0.0203 | 0.0219 | 0.0108 | 0.0110 |
| c=1.25 | 0.0442 | 0.0723 | 0.0217 | 0.0212 | 0.0265 | 0.0267 | 0.0094 |
| c=1.50 | 0.0557 | 0.0352 | 0.0453 | 0.0245 | 0.0186 | 0.0278 | 0.0499 |
| c=1.75 | 0.1124 | 0.1121 | 0.0322 | 0.0291 | 0.0406 | 0.0415 | 0.0337 |
| c=2.00 | 0.0876 | 0.0352 | 0.0518 | 0.0535 | 0.0605 | 0.0645 | 0.0701 |

* values of $\eta_B$ when $g(x)$ is Gamma distribution (shape=2,scale=1)

| n      | 3      | 4      | 5      | 6      | 7      | 8      | 9      |
|--------|--------|--------|--------|--------|--------|--------|--------|
| c=1.00 | 0.0060 | 0.0226 | 0.0272 | 0.0230 | 0.0280 | 0.0087 | 0.0155 |
| c=1.25 | 0.0184 | 0.0424 | 0.0247 | 0.0236 | 0.0215 | 0.0220 | 0.0135 |
| c=1.50 | 0.0672 | 0.0608 | 0.0712 | 0.0422 | 0.0399 | 0.0213 | 0.0233 |
| c=1.75 | 0.0751 | 0.0749 | 0.0853 | 0.0581 | 0.0463 | 0.0315 | 0.0479 |
| c=2.00 | 0.0965 | 0.0494 | 0.0487 | 0.0935 | 0.0384 | 0.0682 | 0.0665 |

